In [10]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import efficientnet_b0,efficientnet_b3,efficientnet_b5, EfficientNet_B0_Weights,EfficientNet_B3_Weights,EfficientNet_B5_Weights
from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.labels_df.iloc[idx, 0])  # First column is file path
        image = Image.open(img_path).convert("RGB")  # Ensure image is in RGB format
        label = self.labels_df.iloc[idx, 1]  # Second column is label

        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
train_csv = "train_set.csv"
test_csv = "test_set.csv"
root_dir = "lung_image_sets"

In [4]:

# Hyperparameters
num_classes = 10  # Change this to your number of classes
batch_size = 32   # Adjust based on your VRAM
learning_rate = 0.001
num_epochs = 10
image_size = 224  # EfficientNet-B0 expects 224x224 images

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
base_folder="./lung_image_sets"

In [6]:
def scan_folder_to_dataframe(base_folder):
    # List to store file paths and labels
    data = []

    # Walk through all subfolders and files in the base folder
    for root, dirs, files in os.walk(base_folder):
        for file in files:
            # Get full file path
            file_path = os.path.join(root, file)
            # Get the label (folder name)
            label = os.path.basename(root)
            # Append file path and label to the data list
            data.append((file_path, label))

    # Create a DataFrame from the data list
    df = pd.DataFrame(data, columns=['FilePath', 'Label'])
    return df

def create_and_split_dataset(base_folder, test_size=0.2):
    # Create dataset
    dataset = scan_folder_to_dataframe(base_folder)

    # Split the dataset into training and testing sets
    train_set, test_set = train_test_split(dataset, test_size=test_size, stratify=dataset['Label'], random_state=42)

    return train_set, test_set

In [12]:
model2 = efficientnet_b3(weights=EfficientNet_B3_Weights.IMAGENET1K_V1)
model3 = efficientnet_b5(weights=EfficientNet_B5_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/efficientnet_b5_lukemelas-1a07897c.pth" to C:\Users\aydhi/.cache\torch\hub\checkpoints\efficientnet_b5_lukemelas-1a07897c.pth
100%|██████████| 117M/117M [01:56<00:00, 1.05MB/s] 


In [13]:

# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet stats
])

train_dataset = CustomImageDataset(csv_file=train_csv, root_dir=root_dir, transform=transform)
test_dataset = CustomImageDataset(csv_file=test_csv, root_dir=root_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Load pretrained EfficientNet-B0
model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)


model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=num_classes),  
)

model = model.to(device)


In [14]:


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)


        outputs = model(images)
        loss = criterion(outputs, labels)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {running_loss / len(train_loader):.4f}")


FileNotFoundError: [Errno 2] No such file or directory: 'lung_image_sets\\lung_image_sets\\lung_n\\lungn3582.jpeg'

: 

In [ ]:

# Testing loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")